In [1]:
# some example data
import statsmodels as sm
import pandas
import numpy as np
from statsmodels.tsa.vector_ar.var_model import VAR
mdata = sm.datasets.macrodata.load_pandas().data

# prepare the dates index
dates = mdata[['year', 'quarter']].astype(int).astype(str)
quarterly = dates["year"] + "Q" + dates["quarter"]
from statsmodels.tsa.base.datetools import dates_from_str
quarterly = dates_from_str(quarterly)

mdata = mdata[['realgdp','realcons','realinv']]
mdata.index = pandas.DatetimeIndex(quarterly)
data = np.log(mdata).diff().dropna()

# make a VAR model
model = VAR(data)

In [2]:
from seq_input import *
from seq_model import *
data_ndarray = data.as_matrix()
train_data, valid_data, test_data = ptb_raw_data(data_ndarray)

(164, 182)


In [3]:
# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import tensorflow as tf
from tensorflow.models.rnn.ptb import reader
import sys, os
from PTBModel import *
  
flags = tf.flags
logging = tf.logging

flags.DEFINE_string(
    "model", "small",
    "A type of model. Possible options are: small, medium, large.")
flags.DEFINE_string("data_path", "../data/PTB_data/",
                    "Where the training/test data is stored.")
flags.DEFINE_string("save_path", "../log/",
                    "Model output directory.")
flags.DEFINE_bool("use_fp16", False,
                  "Train using 16-bit floats instead of 32bit floats")

FLAGS = flags.FLAGS



class TestConfig(object):
    """Tiny config, for testing."""
    init_scale = 0.1
    learning_rate = 1e-1
    max_grad_norm = 1
    num_layers = 1
    num_steps = 2
    hidden_size = 2
    max_epoch = 10
    max_max_epoch = 10
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 2
    vocab_size = 3

def run_epoch(session, model, eval_op=None, verbose=False):
    """Runs the model on the given data."""
    start_time = time.time()
    costs = 0.0
    iters = 0
    state = session.run(model.initial_state)

    fetches = {
        "cost": model.cost,
        "final_state": model.final_state,
    }
    if eval_op is not None:
        fetches["eval_op"] = eval_op

    for step in range(model.input.epoch_size):
        feed_dict = {}
        for i, (c, h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h

        vals = session.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]

        costs += cost
        iters += model.input.num_steps

        if verbose and step % (model.input.epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                  (step * 1.0 / model.input.epoch_size, np.exp(costs / iters),
                   iters * model.input.batch_size / (time.time() - start_time)))

    return costs / iters


def main(_):
    if not FLAGS.data_path:
        raise ValueError("Must set --data_path to PTB data directory")

    raw_data = ptb_raw_data(data_ndarray)#seq raw data
    train_data, valid_data, test_data = raw_data
    config = TestConfig()
    eval_config = TestConfig()
    eval_config.batch_size = 1
    eval_config.num_steps = 1

    with tf.Graph().as_default():
        initializer = tf.random_uniform_initializer(-config.init_scale,
                                                    config.init_scale)
        with tf.name_scope("Train"):
            train_input = PTBInput(config=config, data=train_data, name="TrainInput")
            with tf.variable_scope("Model", reuse=None, initializer=initializer):
                m = PTBModel(is_training=True, config=config, input_=train_input)
            tf.summary.scalar("Training_Loss", m.cost)
            tf.summary.scalar("Learning_Rate", m.lr)

        with tf.name_scope("Valid"):
            valid_input = PTBInput(config=config, data=valid_data, name="ValidInput")
            with tf.variable_scope("Model", reuse=True, initializer=initializer):
                mvalid = PTBModel(is_training=False, config=config, input_=valid_input)
            tf.summary.scalar("Validation_Loss", mvalid.cost)

        with tf.name_scope("Test"):
            test_input = PTBInput(config=eval_config, data=test_data, name="TestInput")
            with tf.variable_scope("Model", reuse=True, initializer=initializer):
                mtest = PTBModel(is_training=False, config=eval_config,
                                 input_=test_input)

        sv = tf.train.Supervisor(logdir=FLAGS.save_path)
        with sv.managed_session() as session:
            for i in range(config.max_max_epoch):
                lr_decay = config.lr_decay ** max(i + 1 - config.max_epoch, 0.0)
                m.assign_lr(session, config.learning_rate * lr_decay)

                print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
                train_err = run_epoch(session, m, eval_op=m.train_op,
                                             verbose=True)
                print("Epoch: %d Train Error: %.3f" % (i + 1, train_err))
                valid_err = run_epoch(session, mvalid)
                print("Epoch: %d Valid Error: %.3f" % (i + 1, valid_err))

            test_err = run_epoch(session, mtest)
            print("Test Error: %.3f" % test_err)

            if FLAGS.save_path:
                print("Saving model to %s." % FLAGS.save_path)
                sv.saver.save(session, FLAGS.save_path, global_step=sv.global_step)


if __name__ == "__main__":
    tf.app.run()


(164, 182)
Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.


Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.


INFO:tensorflow:Model/global_step/sec: 0
Epoch: 1 Learning rate: 0.100
Epoch: 1 Train Error: 0.000
Epoch: 1 Valid Erro: 0.000
Epoch: 2 Learning rate: 0.100
Epoch: 2 Train Error: 0.000
Epoch: 2 Valid Erro: 0.000
Epoch: 3 Learning rate: 0.100
Epoch: 3 Train Error: 0.000
Epoch: 3 Valid Erro: 0.000
Epoch: 4 Learning rate: 0.100
Epoch: 4 Train Error: 0.000
Epoch: 4 Valid Erro: 0.000
Epoch: 5 Learning rate: 0.100
Epoch: 5 Train Error: 0.000
Epoch: 5 Valid Erro: 0.000
Epoch: 6 Learning rate: 0.100
Epoch: 6 Train Error: 0.000
Epoch: 6 Valid Erro: 0.000
Epoch: 7 Learning rate: 0.100
Epoch: 7 Train Error: 0.000
Epoch: 7 Valid Erro: 0.000
Epoch: 8 Learning rate: 0.100
Epoch: 8 Train Error: 0.000
Epoch: 8 Valid Erro: 0.000
Epoch: 9 Learning rate: 0.100
Epoch: 9 Train Error: 0.000
Epoch: 9 Valid Erro: 0.000
Epoch: 10 Learning rate: 0.100
Epoch: 10 Train Error: 0.000
Epoch: 10 Valid Erro: 0.000
INFO:tensorflow:Error reported to Coordinator: <type 'exceptions.NameError'>, global name 'test_perplexity

INFO:tensorflow:Model/global_step/sec: 0
INFO:tensorflow:Error reported to Coordinator: <type 'exceptions.NameError'>, global name 'test_perplexity' is not defined


NameError: global name 'test_perplexity' is not defined